# The wekeo provider in EODAG

This tutorial will show you how use EODAG to download data from [WEkEO](https://www.wekeo.eu/) using `eodag`. WEkEO offers a wide range of Copernicus and Sentinel products.

## Registration

For both, searching and downloading data, authentication is required. Therefore, you first have to create an account on [https://www.wekeo.eu/](https://www.wekeo.eu/) to get a `username` and a `password` which have to be added to the EODAG configuration (see [Configure EODAG](https://eodag.readthedocs.io/en/stable/getting_started_guide/configure.html#configure)). After registration you also have to accept the Terms and Conditions for the datasets you want to use to be able to retrieve data from EODAG. This has to be done once per user and licence by executing the following code where licence_name has to be set to the desired licence (see list below):

In [ ]:
import requests
from eodag import EODataAccessGateway

# instantiate EODAG API
dag = EODataAccessGateway()

# Authenticate on wekeo
auth = dag._plugins_manager.get_auth_plugin("wekeo").authenticate()
licence_name = "EUMETSAT_Copernicus_Data_Licence"

# Use your authentication to accept Terms and Conditions
response = requests.put(
    f"https://gateway.prod.wekeo2.eu/hda-broker/api/v1/termsaccepted/{licence_name}",
    auth=auth,
)

Depending on which data you want to use, different licences have to be accepted. If you want to use all the product types available in EODAG, the following licences have to be accepted:
      
* EUMETSAT_Copernicus_Data_Licence
* Copernicus_Land_Monitoring_Service_Data_Policy
* Copernicus_Sentinel_License
* Copernicus_ECMWF_License
* Copernicus_Marine_Service_Product_License
* Copernicus_DEM_Instance_COP-DEM-GLO-30-F_Global_30m
* Copernicus_DEM_Instance_COP-DEM-GLO-90-F_Global_90m

## Search and download data

Many product types are available for `wekeo` provider:

In [2]:
wekeo_product_types = dag.list_product_types("wekeo")
len(wekeo_product_types)

251

In [3]:
[pt["ID"] for pt in wekeo_product_types]

['CAMS_EAC4',
 'CAMS_EAC4_MONTHLY',
 'CAMS_EU_AIR_QUALITY_FORECAST',
 'CAMS_EU_AIR_QUALITY_RE',
 'CAMS_GAC_FORECAST',
 'CAMS_GLOBAL_EMISSIONS',
 'CAMS_GREENHOUSE_EGG4',
 'CAMS_GREENHOUSE_EGG4_MONTHLY',
 'CAMS_GREENHOUSE_INVERSION',
 'CAMS_GRF',
 'CAMS_GRF_AUX',
 'CAMS_SOLAR_RADIATION',
 'CLMS_CORINE',
 'CLMS_GLO_DMP_333M',
 'CLMS_GLO_FAPAR_333M',
 'CLMS_GLO_FCOVER_333M',
 'CLMS_GLO_GDMP_333M',
 'CLMS_GLO_LAI_333M',
 'CLMS_GLO_NDVI_1KM_LTS',
 'CLMS_GLO_NDVI_333M',
 'CMEMS_GLOBAL_BGC__anfc_0_25deg_static_202311',
 'CMEMS_GLOBAL_BGC__bio_anfc_0_25deg_P1D-m_202311',
 'CMEMS_GLOBAL_BGC__bio_anfc_0_25deg_P1M-m_202311',
 'CMEMS_GLOBAL_BGC__car_anfc_0_25deg_P1D-m_202311',
 'CMEMS_GLOBAL_BGC__car_anfc_0_25deg_P1M-m_202311',
 'CMEMS_GLOBAL_BGC__co2_anfc_0_25deg_P1D-m_202311',
 'CMEMS_GLOBAL_BGC__co2_anfc_0_25deg_P1M-m_202311',
 'CMEMS_GLOBAL_BGC__nut_anfc_0_25deg_P1D-m_202311',
 'CMEMS_GLOBAL_BGC__nut_anfc_0_25deg_P1M-m_202311',
 'CMEMS_GLOBAL_BGC__optics_anfc_0_25deg_P1D-m_202311',
 'CMEMS_GLOB

Search for data on one these product types:

In [4]:
products, total_count = dag.search(
    productType="GLOFAS_FORECAST", 
    start="2023-01-01", 
    end="2023-01-02", 
    geom=[0.25, 43.2, 2.8, 43.9],
    provider="wekeo"
)
products

SearchResult([EOProduct(id=4bd37b17b45d146f378e349ab3f90464, provider=wekeo)])

Download found product to `/tmp`:

In [5]:
path = products[0].download(outputs_prefix="/tmp")
path

0.00B [00:00, ?B/s]

Downloaded product is not a Zip File. Please check its file type before using it


'/tmp/4bd37b17b45d146f378e349ab3f90464'

## The WEkEO data viewer

If you go to the WEkEO [homepage](https://www.wekeo.eu) and click on "Data" in the top menu, you can search for products online. You can choose a product type (called dataset in WEkEO) by clicking on the "+" button next to "Layers" on the left side. Then you can choose some filter parameters and create a data request job which will show up in the next tab where the products can be downloaded. The data viewer currently still uses the old version of the API where data requests have to be created to search product. In contrast, the latest version of EODAG already uses the new API (https://gateway.prod.wekeo2.eu/hda-broker/api/v1) where a direct search without data requests is possible.